<a href="https://colab.research.google.com/github/tjwei/taishin20190827/blob/master/CF/knn_cust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 用 KNN 的方式做推薦

台新分析開發環境案客戶資料
https://drive.google.com/drive/u/2/folders/1AAXWPcB9xiDe_qX87ce5XL-utkASWxRe

請先連到這個分享資料夾，並且按下右鍵，加入你的 Google 雲端硬碟


In [1]:
# 這個是授權存取你的 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 確認真的連上了資料
!ls drive/My\ Drive/台新分析開發環境案客戶資料

'Copy of 0606_口述台新資料欄位.m4a'
'Copy of 0610_嘉駿寄來的欄位內容.png'
'Copy of 0621_data_original.zip'
'Copy of profile.csv'
'Copy of profile.gsheet'
'Copy of result_y1.csv'
'Copy of result_y1.gsheet'
'Copy of result_y2.csv'
'Copy of sr_1.csv'
'Copy of sr_2.csv'
'Copy of sr_3.csv'
'Copy of sr_4.csv'
'Copy of status.csv'
'Copy of visa-merchant-data-standards-manual_用來對照sr_1.csv中的mcc欄位.pdf'
'Copy of zip 對照表.xlsx'
'Copy of 檔案欄位說明_MOOCS.XLSX'


In [0]:
# 正式開始
import pandas as pd
import os.path
import numpy as np

In [0]:
DATA_DIR='drive/My Drive/台新分析開發環境案客戶資料'
sr_1_fn = os.path.join(DATA_DIR, 'Copy of sr_1.csv')
with open(sr_1_fn, encoding='cp950', errors='replace') as f:
    df = pd.read_csv(f, nrows=1000000, usecols=['CUST_NO', 'TXN_DESC'])

In [5]:
df.head()

,CUST_NO,TXN_DESC
0,564755,全聯台中四平分公司 TAICHU/TW
1,564755,全聯台中四平分公司 TAICHU/TW
2,6185152,健身工廠九如廠 Kaohsi/TW
3,6185152,富邦ｍｏｍｏ－ＥＣ TAIPEI/TW
4,5318034,國泰人壽保費部 第 12/12 期 /TW


In [6]:
df.TXN_DESC.nunique()

204796

In [7]:
TXN = df.TXN_DESC.str.extract(r'([^\s０－９（５１２）]+)')
df['TXN_DESC'] = TXN
df.TXN_DESC.nunique()

41704

In [8]:
# 看一下使用頻率
txn_counts = df.TXN_DESC.value_counts()
txn_counts.quantile(np.arange(.5,1,.01))

0.50      2.0
0.51      2.0
0.52      2.0
0.53      2.0
0.54      2.0
0.55      2.0
0.56      2.0
0.57      2.0
0.58      2.0
0.59      2.0
0.60      2.0
0.61      2.0
0.62      2.0
0.63      3.0
0.64      3.0
0.65      3.0
0.66      3.0
0.67      3.0
0.68      3.0
0.69      3.0
0.70      3.0
0.71      4.0
0.72      4.0
0.73      4.0
0.74      4.0
0.75      5.0
0.76      5.0
0.77      5.0
0.78      6.0
0.79      6.0
0.80      6.0
0.81      7.0
0.82      7.0
0.83      8.0
0.84      9.0
0.85     10.0
0.86     11.0
0.87     13.0
0.88     14.0
0.89     16.0
0.90     19.0
0.91     22.0
0.92     26.0
0.93     31.0
0.94     38.0
0.95     48.0
0.96     63.0
0.97     84.0
0.98    118.0
0.99    218.0
Name: TXN_DESC, dtype: float64

In [9]:
# 只留下交易一定次數以上的商店
good_txn = txn_counts[txn_counts>50]
#df = df.loc[df.TXN_DESC.isin(good_txn.index)]
df.drop(df.loc[~df.TXN_DESC.isin(good_txn.index)].index, inplace=True)
df.TXN_DESC.nunique()

2016

In [10]:
# 看看用戶交易次數
cust_counts = df.CUST_NO.value_counts()
cust_counts.quantile(np.arange(0,1,.01))

0.00     1.0
0.01     1.0
0.02     1.0
0.03     1.0
0.04     1.0
0.05     1.0
0.06     1.0
0.07     1.0
0.08     1.0
0.09     1.0
0.10     1.0
0.11     1.0
0.12     1.0
0.13     1.0
0.14     1.0
0.15     1.0
0.16     1.0
0.17     1.0
0.18     1.0
0.19     1.0
0.20     2.0
0.21     2.0
0.22     2.0
0.23     2.0
0.24     2.0
0.25     2.0
0.26     2.0
0.27     2.0
0.28     2.0
0.29     2.0
        ... 
0.70     7.0
0.71     7.0
0.72     8.0
0.73     8.0
0.74     8.0
0.75     8.0
0.76     9.0
0.77     9.0
0.78     9.0
0.79    10.0
0.80    10.0
0.81    10.0
0.82    11.0
0.83    11.0
0.84    11.0
0.85    12.0
0.86    12.0
0.87    13.0
0.88    14.0
0.89    14.0
0.90    15.0
0.91    16.0
0.92    17.0
0.93    18.0
0.94    19.0
0.95    21.0
0.96    23.0
0.97    25.0
0.98    29.0
0.99    37.0
Name: CUST_NO, Length: 100, dtype: float64

In [11]:
good_cust = cust_counts[cust_counts>15]
#df = df.loc[df.CUST_NO.isin(good_cust.index)]
df.drop(df.loc[~df.CUST_NO.isin(good_cust.index)].index, inplace=True)
print(df.TXN_DESC.nunique())
print(df.CUST_NO.nunique())

2016
11620


In [12]:
# pivot dataframe, 變成我們要的矩陣
df = df.pivot_table(index='CUST_NO', columns='TXN_DESC', aggfunc=len, fill_value=0 )
df

TXN_DESC,AGODA,AGODA.COM,AIR,AIRASIA,AIRBNB,ALI*www.taobao.com,ALIPAY*TAOBAO.COM,AMAZON,AMAZON.CO.JP,APPLE,ATM,Amazon,AmazonPrime,BIC,Buyee,CATHAYPA,CMPCYBERSTEP,Ctrip,DAIKOKUDRUG,DDITSERVICES.COM,DFS,DMG,DON,DONKI,EXPEDIA,FACEBK,GALALUZ,GLOBALBLUE.COM/RULES,GMAKET(HAEOI),GMAKET(HAEOIDCC),GOOGLE,GOOGLE*BNEI,GOOGLE*CAMEL,GOOGLE*DA,GOOGLE*DIANCHU,GOOGLE*DTSH5,GOOGLE*ENJOY,GOOGLE*EYOU,GOOGLE*EYOUNI,GOOGLE*FGO,...,ＴｉｇｅｒＣｉｔｙ,ＵＮＩＱＬＯ,ＷＯＲＬＤＧＹＭ,ＷＯＲＬＤＧＹＭ健身,ＷＯＲＬＤＧＹＭ健身俱樂部,ＷｏｒｌｄＧｙｍ三重店,ＷｏｒｌｄＧｙｍ公館店,ＷｏｒｌｄＧｙｍ台中學府店,ＷｏｒｌｄＧｙｍ台中西屯店,ＷｏｒｌｄＧｙｍ大直店,ＷｏｒｌｄＧｙｍ崇德店,ＷｏｒｌｄＧｙｍ板橋雙十店,ＷｏｒｌｄＧｙｍ高雄中華店,ＷｏｒｌｄＧｙｍ黎明店,ＹｉＰａｙ,ＹｏｕＢｉｋｅ公共自行車租賃,ＺＡＲＡ,ｂｏｓｓｉｎｉ,ｃａｍａ台北和平店,ｃａｍａ台北奇岩店,ｃａｍａ台北承德店,ｃａｍａ台南西門店,ｃａｍａ天母蘭雅店,ｃａｍａ忠孝復興店,ｃａｍａ忠孝敦化店,ｃａｍａ永和中山店,ｅｍｏｎ依夢內衣,ｆｒｉＤａｙ購物,ｆｒｉＤａｙ購物ｘＧ,ｆｒｉＤａｙ錢包,ｈｏｎｅｓｔｂｅｅ,ｉｎ８,ｌａｔｉｖ,ｍｙｆｏｎｅ購物,ｍｙｐａｙ,ｒａｉｌｗａｙ,ｔｉｘＣｒａｆｔ售票網,ｕｄｎ售票網,ｕｄｎ買東西,ｗｉｆｉｈｅｒｏ漫遊超人
CUST_NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
44429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49499,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53517,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54748,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56868,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# 用來方便印出使用者資料
def print_cust(n):
    print(df.iloc[n][df.iloc[n] > 0])

In [18]:
for i in range(5):
  print_cust(i)
  print()

TXN_DESC
GOOGLE*USERJOY    48
Name: 44429, dtype: int64

TXN_DESC
全家便利商店       2
全聯中港分公司      1
全聯林口信義分公司    5
富邦人壽         1
富邦人壽應繳年月     1
支付平台         1
美廉社          5
Name: 46973, dtype: int64

TXN_DESC
ITUNES.COM/BILL    4
eTag               1
中油                 1
台茂購物中心             1
康福旅行社股份有限公         2
東森得易購股份有限公         1
神腦國際               1
繳交                 4
繳交台北市停車費           3
繳交桃園市停車費           3
繳交欣桃瓦斯費            1
Name: 48428, dtype: int64

TXN_DESC
eTag     3
中油      15
Name: 49499, dtype: int64

TXN_DESC
杏一醫療用品ＡＢ     1
街口電支        21
Name: 50758, dtype: int64



In [21]:
# sklearn NN
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='correlation', algorithm='brute')
model_knn.fit(df)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='correlation',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [19]:
# 1 號使用者
print_cust(1)

TXN_DESC
全家便利商店       2
全聯中港分公司      1
全聯林口信義分公司    5
富邦人壽         1
富邦人壽應繳年月     1
支付平台         1
美廉社          5
Name: 46973, dtype: int64


In [23]:
# 找到跟 1 號使用者最相似的 5 個人
distances, indices = model_knn.kneighbors(df.iloc[1:2], n_neighbors=6)
for i in range(1, 6):
    print(distances[0][i])
    print_cust(indices[0][i])
    print()

0.32380867739930574
TXN_DESC
ITUNES.COM/BILL     4
全家便利商店              5
全聯汐止康寧分公司           1
台灣大車隊               3
好樂迪                 1
美廉社                41
頂好Ｗｅｌｌｃｏｍｅ          2
ＯＫ超商                2
Name: 635428, dtype: int64

0.3303470303549835
TXN_DESC
三創生活              1
全家便利商店            5
台灣麥當勞             3
太平洋崇光百貨股份有限公司     1
屈臣氏Ｓ              2
微風台北車站美食          1
美廉社              12
美麗華百樂園            1
萊爾富               1
ＯＫ超商              1
Name: 9727234, dtype: int64

0.3313509918989438
TXN_DESC
全家便利商店     1
美廉社       20
Name: 10406581, dtype: int64

0.3316309316518541
TXN_DESC
全家便利商店         2
全聯平鎮中豐分公司      1
定食８            1
悠遊卡自動加值        3
統一精工股份有限公司     1
美廉社           22
ＣＯＣＯ都可         1
Name: 10716153, dtype: int64

0.3389045440877435
TXN_DESC
FACEBK           2
GOOGLE*NEXON     3
中油               7
代繳遠傳帳單           2
全家便利商店           8
國泰人壽             1
國泰人壽保費部          3
潤泰旭展股份有限公司       1
美廉社             22
Name: 10358584, dtype: int64

